# Image classification - Kaggle

In [15]:
from fastai.vision import *
from fastai.metrics import accuracy
import time
print("Import of requried Pkgs - Done")

Import of requried Pkgs - Done


# Copy data from GDrive to Notebook

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [24]:
!ls drive/'My Drive'/images/

sample_submission.csv  test  train


In [13]:
# !mkdir -p 'drive/data'  # error - mkdir: cannot create directory ‘drive/data’: Operation not supported
!mkdir -p data

mkdir: cannot create directory ‘drive/data’: Operation not supported


In [19]:
start_time = time.time()
!cp -R drive/'My Drive'/images/train/train data/train
print("--- %s seconds ---" % ((time.time() - start_time)/60))

^C
--- 8.426594285170237 seconds ---


In [0]:
!cp drive/'My Drive'/images/sample_submission.csv data/sample_submission.csv

In [20]:
!ls

data  drive  dsnet-kaggledays-hackathon.zip  sample_data


In [22]:
!tar -xvf datakaggle.tar

datakaggle/train/train/Edvard_Munch/Edvard_Munch_4.jpg
datakaggle/test/test/892dbf13-27d0-443e-a6e3-c0b81f2c0c57.jpg
datakaggle/train/train/Hieronymus_Bosch/Hieronymus_Bosch_85.jpg
datakaggle/test/test/726377f4-0f12-4205-a5c1-507dba76254b.jpg
datakaggle/train/train/Francisco_Goya/Francisco_Goya_228.jpg
datakaggle/test/test/a501b83c-e241-4611-92ae-9767b49e8be1.jpg
tar: Unexpected EOF in archive
tar: rmtlseek not stopped at a record boundary
tar: Error is not recoverable: exiting now


In [23]:
help(fit_one_cycle)

Help on function fit_one_cycle in module fastai.train:

fit_one_cycle(learn:fastai.basic_train.Learner, cyc_len:int, max_lr:Union[float, Collection[float], slice]=slice(None, 0.003, None), moms:Tuple[float, float]=(0.95, 0.85), div_factor:float=25.0, pct_start:float=0.3, final_div:float=None, wd:float=None, callbacks:Union[Collection[fastai.callback.Callback], NoneType]=None, tot_epochs:int=None, start_epoch:int=None) -> None
    Fit a model following the 1cycle policy.



## Data

In [0]:
# # Copy for FastAI
# !mkdir -p data
# !cp -R ../input/dsnet-kaggledays-hackathon/train/train data/train
# # !cp -R ../input/dsnet-kaggledays-hackathon/test/test data/test
# !cp ../input/dsnet-kaggledays-hackathon/sample_submission.csv data/sample_submission.csv

In [0]:
!ls data

In [0]:
DATA_DIR = Path('data')

In [0]:
?Path

In [0]:
DATA_DIR/'train'

In [0]:
bs = 64

In [0]:
data = ImageDataBunch.from_folder(DATA_DIR, valid_pct=0.2, ds_tfms=get_transforms(flip_vert="True"), bs=180, size=64)

In [0]:
?get_tranforms()

In [0]:
data.show_batch(rows=3, figsize=(7,6))

## Model

In [0]:
learn = cnn_learner(data, models.resnet18, pretrained=False, metrics=accuracy)

In [0]:
learn.model

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

In [0]:
learn.fit_one_cycle(4, 3e-3)

In [0]:
# Save the model
learn.save('simple-model')

In [0]:
# Load the model
learn.load('simple-model');

In [0]:
!cp -R ../input/dsnet-kaggledays-hackathon/test/test data/test

## Inference

In [0]:
# Load submission file
sample_df = pd.read_csv(DATA_DIR/'sample_submission.csv')
sample_df.head()

In [0]:
# Generate test predictions
learn.data.add_test(ImageList.from_df(sample_df,DATA_DIR,folder='test'))

In [0]:
# Load up submission file
preds,y = learn.get_preds(DatasetType.Test)

In [0]:
# Convert predictions to classes
pred_classes = [data.classes[c] for c in list(preds.argmax(dim=1).numpy())]
pred_classes[:10]

In [0]:
# Add the prediction
sample_df.predicted_class = pred_classes
sample_df.head()

In [0]:
# Save the submission file
sample_df.to_csv('submission.csv',index=False)

from IPython.display import FileLink
FileLink('submission.csv')

In [0]:
# Clean up (for commit)
!cp -R data/models models # Move the models out
!rm -rf data # Delete the data